Here's a content based recommendation system for the anime using the nearest neighbor approach, that has surprisingly good results compared to the straight-forward approach using anime genres, anime type, rating, episode number and member numbers as features(along with some preprocessing of the text data and feature scaling). It's returning 2nd season of the animes for the queries so far. It's also good at recommending similar movies for movie queries and anime TV shows for the TV show queries.


![](http://2.bp.blogspot.com/-JkQezo4c7LQ/VN1QJdAp_kI/AAAAAAAAfQs/e_zu299T8bI/s1600/anime_wallpaper_by_m)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
%matplotlib inline

In [ ]:
anime = pd.read_csv("../input/anime.csv")

In [ ]:
anime.head()

# Data Preprocessing : 

Many animes have unknown number of episodes even if they have similar rating. On top of that many super popular animes such as Naruto Shippuden, Attack on Titan Season 2 were ongoing when the data was collected, thus their number of episodes was considered as "Unknown". For some of my favorite animes I've filled in the episode numbers manually. For the other anime's, I had to make some educated guesses. Changes I've made are :

1. Animes that are grouped under Hentai Categories generally have 1 episode in my experience. So I've filled the unknown values with 1.

2. Animes that are grouped are "OVA" stands for "Original Video Animation". These are generally one/two episode long animes(often the popular ones have 2/3 episodes though), but I've decided to fill the unknown numbers of episodes with 1 again.

3. Animes that are grouped under "Movies" are considered as '1' episode as per the dataset overview goes.

4. For all the other animes with unknown number of episodes, I've filled the na values with the median which is 2.  


In [ ]:
anime.loc[(anime["genre"]=="Hentai") & (anime["episodes"]=="Unknown"),"episodes"] = "1"
anime.loc[(anime["type"]=="OVA") & (anime["episodes"]=="Unknown"),"episodes"] = "1"

anime.loc[(anime["type"] == "Movie") & (anime["episodes"] == "Unknown")] = "1"

In [ ]:
known_animes = {"Naruto Shippuuden":500, "One Piece":784,"Detective Conan":854, "Dragon Ball Super":86,
                "Crayon Shin chan":942, "Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25}

In [ ]:
for k,v in known_animes.items():    
    anime.loc[anime["name"]==k,"episodes"] = v

In [ ]:
anime["episodes"] = anime["episodes"].map(lambda x:np.nan if x=="Unknown" else x)

In [ ]:
anime["episodes"].fillna(anime["episodes"].median(),inplace = True)

### Rating 

Many animes have unknown ratings. These were filled with the median of the ratings.

In [ ]:
anime["rating"] = anime["rating"].astype(float)

In [ ]:
anime["rating"].fillna(anime["rating"].median(),inplace = True)

### Type 

Type category differentiates between movies, music, TV shows(regular anime episodes), OVA/ONA etc. These are categorical variables so I used ```pd.get_dummies``` to convert them to dummy variables.

In [ ]:
pd.get_dummies(anime[["type"]]).head()

### Members

Just converted the strings to float.

In [ ]:
anime["members"] = anime["members"].astype(float)

In [ ]:
# anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),pd.get_dummies(anime[["type"]]),anime[["rating","episodes"]]],axis=1)

# Feature Selection and Preprocessing



Episode numbers, members and rating are different from categorical variables and very different in values. Rating ranges from 0-10 in the dataset while the episode number can be even 800+ episodes long when it comes to long running popular animes such as One Piece, Naruto etc. So I ended up using ```sklearn.preprocessing.MaxAbsScaler``` as it preserves the sparsity while scaing the values from 0-1.

In [ ]:
anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),pd.get_dummies(anime[["type"]]),anime[["rating"]],anime[["members"]],anime["episodes"]],axis=1)

In [ ]:
anime["name"] = anime["name"].map(lambda name:re.sub('[^A-Za-z0-9]+', " ", name))

In [ ]:
anime_features.head()

In [ ]:
anime_features.columns

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
max_abs_scaler = MaxAbsScaler()
anime_features = max_abs_scaler.fit_transform(anime_features)

# KNN for finding similar animes

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(anime_features)

In [ ]:
distances, indices = nbrs.kneighbors(anime_features)

# Query examples and helper functions 

In [ ]:
def get_index_from_name(name):
    return anime[anime["name"]==name].index.tolist()[0]
    

In [ ]:
get_index_from_name("Naruto")

In [ ]:
distances[841]

In [ ]:
indices[841]

Many anime names have not been documented properly and in many cases the names are in Japanese instead of English and the spelling is often different. For that reason I've also created another helper function ```get_id_from_partial_name``` to find out ids of the animes from part of names.

In [ ]:
all_anime_names = list(anime.name.values)

In [ ]:
def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name,all_anime_names.index(name))

In [ ]:
""" print_similar_query can search for similar animes both by id and by name. """

def print_similar_animes(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.ix[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(anime.ix[id]["name"])

# Query Examples 

<b> Naruto </b>
    
Naruto is a shounen manga and Naruto Shippuden, the 1st neighbor is the closest to it as its the second season. Even if Naruto has 220 episodes and Naruto Shippuden has 500 episodes, it appears that the model was able to find out the second season. I've watchd Katekyo Hitman Reborn too and its very similar to Naruto(I'm a big fan of both) in the story structure. Bleach, Dragon Ball Z are also long standing animes with under the same action,comedy, Shounen genre.

![](http://s1.picswalls.com/wallpapers/2015/09/27/naruto-wallpaper_104044786_274.jpg)

In [ ]:
print_similar_animes(query="Naruto")

<b> Noragami(Stray God).</b>

Noragami is a supernatural manga featuring Japanese Shinto Gods. Noragami Aragoto is the 2nd season. I've not seen the other ones, but they seem to be very highly rated too.

![](http://thisisanothercastle.files.wordpress.com/2014/04/noragami-title2.jpg)

In [ ]:
print_similar_animes("Noragami")

<b>Mushishi, Gintama, Fairy Tail</b> 

I just checked Mushishi and Gintama because my sister asked me to, but looks like we are getting pretty good results. Fairy Tail is about a magical guild, the 1st neighbor is Fairy Tail 2014, when the anime was re-made. The other neighbors seem to be magical anime's too.

In [ ]:
print_similar_animes("Mushishi")

In [ ]:
print_similar_animes("Gintama")

In [ ]:
print_similar_animes("Fairy Tail")

In [ ]:
get_id_from_partial_name("Naruto")

In [ ]:
print_similar_animes(id=719)

<b>3.</b> Kimi no Na wa (Your Name)



In [ ]:
print_similar_animes("Kimi no Na wa ")

Kimi No Na Wa is a super popular movie that won many awards. All the recommendations for this movie is really good. I've seen Harmonie and Hotarubi no Mori e and can absolutely say both are amazing. The 1st recommendation also seems highly rated in IMDB and a fantasy film. Air + Clannad is also highly rated. 

See the wallpaper from Your name  vs the 1st choice "Kokoro ga Sakebitagatterunda". Both are school life based fantasy films focusing on romance.


![](https://images-na.ssl-images-amazon.com/images/M/MV5BODRmZDVmNzUtZDA4ZC00NjhkLWI2M2UtN2M0ZDIzNDcxYThjL2ltYWdlXkEyXkFqcGdeQXVyNTk0MzMzODA@._V1_UY1200_CR90,0,630,1200_AL_.jpg)


![](https://i.ytimg.com/vi/fnD8ABI4JrQ/maxresdefault.jpg)

# Conclusion

It appears the model is returning TV shows for TV shows and movies for movies. I'm pretty much happy with the model despite the simple approach. Next time I'll try to implement collaborative filtering for sure, even if this time we've focused on  the properties of the anime's. 